In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Reddit Post Analysis').getOrCreate()


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [2]:
df = spark.read.option("header", True) \
    .option("inferSchema", True) \
    .option("multiLine", True) \
    .option("quote", '"') \
    .option("escape", '"') \
    .option("mode", "DROPMALFORMED") \
    .csv('reddit_posts.csv')

df_clean = df.filter("id IS NOT NULL AND subreddit IS NOT NULL AND title IS NOT NULL and id LIKE '1%'")
df_clean.createOrReplaceTempView("posts")

df_clean.show(5)


+-------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+---------------+---------+---------+--------------------+---------------------+--------------+-------------+
|     id|           timestamp|       timestamp_est|        subreddit|               title|         description|            keywords|score_immediate|score_3hr|score_8hr|         top_comment|top_comment_sentiment|post_sentiment|     Category|
+-------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+---------------+---------+---------+--------------------+---------------------+--------------+-------------+
|1jxwrkv|2025-04-13 01:41:...|2025-04-12 21:41:...|NoStupidQuestions|Do fat/obese/over...|or does the same ...|['fat', 'obese', ...|              1|        0|        0|I grew up very sk...|             Positive|       Neutral|    Education|
|1jxwrk8|2025-04-13 01:41:...|2025-0

In [3]:
spark.sql("""
select category, count(*) from posts group by 1 ;

""").show() # people post FAR more on reddit about their relationships than anything else. people love to yap. interpersonals matter

+-------------+--------+
|     category|count(1)|
+-------------+--------+
|    Education|    1429|
|Entertainment|     613|
|      Finance|     729|
|Relationships|    3229|
|         Tech|    1009|
|    Wellbeing|    1274|
|        Humor|     639|
|         News|     655|
+-------------+--------+



In [4]:
spark.sql("""
select count(*) from posts;

""").show() # 30 rows were corrupt so after dropping (assume MCAR) we're left with that number of posts and rows

+--------+
|count(1)|
+--------+
|    9577|
+--------+



In [5]:
spark.sql("""
select category, avg(score_3hr), avg(score_8hr) from posts group by 1 order by 2 desc ;

""").show() # relationships (despite being posted on a ton) threads get like no likes on posts. people use reddit primarily to laugh (humor) and interestingly, for news. this is not traditional news outlet but people use it for their news


+-------------+------------------+------------------+
|     category|    avg(score_3hr)|    avg(score_8hr)|
+-------------+------------------+------------------+
|        Humor| 378.2396551724138|          1093.875|
|         News| 339.9567387687188|  676.603139013453|
|      Finance| 90.58233532934132|126.70742358078603|
|    Education| 80.42262372348782| 190.8480603448276|
|Entertainment|36.012727272727275| 87.48556430446195|
|         Tech| 28.76816239316239|39.439169139465875|
|    Wellbeing|25.212094653812446| 94.61864406779661|
|Relationships| 7.774331550802139|12.768020304568529|
+-------------+------------------+------------------+



In [6]:
spark.sql("""
SELECT category,
       AVG(CASE WHEN post_sentiment = 'Positive' THEN score_3hr END) AS avg_score_positive,
       AVG(CASE WHEN post_sentiment = 'Neutral' THEN score_3hr END) AS avg_score_neutral,
       AVG(CASE WHEN post_sentiment = 'Negative' THEN score_3hr END) AS avg_score_negative
FROM posts
GROUP BY category
ORDER BY category;
""").show()
# in 6/8 categories, the negative posts do far better than the neutral or positive posts

+-------------+------------------+------------------+------------------+
|     category|avg_score_positive| avg_score_neutral|avg_score_negative|
+-------------+------------------+------------------+------------------+
|    Education| 399.6496815286624|32.840966921119595|41.878787878787875|
|Entertainment| 37.65983606557377|24.940425531914894| 67.95714285714286|
|      Finance| 99.13793103448276| 61.57707509881423| 111.7378277153558|
|        Humor| 354.6456692913386| 329.1105527638191|453.26237623762376|
|         News|142.95744680851064|100.79389312977099|  472.531914893617|
|Relationships| 4.685258964143427|11.899103139013453| 7.269449715370019|
|         Tech| 4.526153846153846|19.957013574660632|  98.4319526627219|
|    Wellbeing|            14.375|              10.6|31.729333333333333|
+-------------+------------------+------------------+------------------+



In [8]:
spark.sql("""
SELECT title, subreddit, description, post_sentiment
FROM posts
WHERE score_3hr > 10000
ORDER BY score_3hr DESC
""").show(truncate=False)

# Example top post:
# Zelenskyy on Russian attack on Sumy: 'Only scum can act like this'
# Never interrupt your enemy while he is making a mistake.
# Funny post: https://www.reddit.com/r/funny/comments/1jyc8gi/theres_always_one/


+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+-----------+--------------+
|title                                                                                                                                                                                                                                                                                                     |subreddit    |description|post_sentiment|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+--

In [1]:
spark.sql("""

select distinct category from posts;
""").show()

NameError: name 'spark' is not defined

In [ ]:
spark.sql("""


""").show()

In [1]:
spark.sql("""


""").show()

NameError: name 'spark' is not defined

In [2]:
pip install streamlit pyspark pandas


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install praw


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install google-cloud-pubsub


   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.5/4.3 MB 1.5 MB/s eta 0:00:03
   --------- ------------------------------ 1.0/4.3 MB 1.7 MB/s eta 0:00:02
   ------------ --------------------------- 1.3/4.3 MB 1.8 MB/s eta 0:00:02
   ----------------- ---------------------- 1.8/4.3 MB 1.9 MB/s eta 0:00:02
   ---------------------- ----------------- 2.4/4.3 MB 2.0 MB/s eta 0:00:01
   -------------------------- ------------- 2.9/4.3 MB 2.0 MB/s eta 0:00:01
   ------------------------------- -------- 3.4/4.3 MB 2.1 MB/s eta 0:00:01
   ------------------------------------ --- 3.9/4.3 MB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 4.3/4.3 MB 2.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.3
    Uninstalling protobuf-4.25.3:
      Successfully uninstalled protobuf-4.25.3
Note: you may n